In [1]:
cd ../TileEM/

/srv/data/home/jlee782/crowd-seg/analysis/TileEM


In [2]:
%pylab inline
%load_ext autoreload
%autoreload 2
from paper_visualizations import * 
from PixelEM import * 
from test_sample import test_sample_obj
import json
import pandas as pd
object_lst = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 36, 37, 38, 39, 42, 43, 44, 45, 46, 47]

Populating the interactive namespace from numpy and matplotlib


### Create Indicator Matrix for Pixel-based tiles

Generated voted_workers_mask.pkl which is a image map containing a list of values , since indMat is too large to store since too many pixels, we won't try to store it, instead we will use the voted_workers_mask.pkl as a lookup table

# all_tile_info.csv
- [batch, objid, workerID, tileID, tile area, tile overlap with GT]

In [3]:
from runAreaMask import * 
from skimage import measure
from matplotlib import _cntr as cntr

In [4]:
def pix_intersection_area(mask1,mask2):
    return len(np.where((mask1==mask2)&(mask1==1)&(mask2==1))[0])

In [53]:
tile_tbl =[]
for batch in tqdm(['5workers_rand0','10workers_rand0','15workers_rand0','20workers_rand0','25workers_rand0','30workers_rand0']):
    for objid in object_lst:
#         print "batch: {}; obj{}".format(batch,objid)
        gt = get_gt_mask(objid)
        mega_mask = pkl.load(open('pixel_em/{}/obj{}/mega_mask.pkl'.format(batch,objid)))
        unique_tile_values = np.unique(mega_mask)
        # Tile loop 
        tarea_lst = []
        tidx=0
        for tile_value in unique_tile_values[1:]: #exclude 0
            blobs = mega_mask==tile_value
            blobs_labels = measure.label(blobs,background=0)
            for bidx in np.unique(blobs_labels)[1:]:
                tile = blobs_labels==bidx
                tarea = mask_area(tile)
                tarea_lst.append(tarea)
                overlap_area  = pix_intersection_area(tile,gt)
                tile_tbl.append([batch,objid,tidx,tarea,overlap_area])
                tidx+=1
        outside_area  = np.product(np.shape(gt))-sum(tarea_lst)
        tile_tbl.append([batch,objid,tidx+1,outside_area,0])

In [57]:
tile_tbl = pd.DataFrame(tile_tbl,columns=["batch","objid","tile id","tile area", "gt overlap area"])
dedup_tile_tbl = tile_tbl.drop_duplicates()
dedup_tile_tbl.to_csv("all_pixel_tile_info.csv")

# Worker Tile Table 

In [ ]:
worker_tile_tbl = []
for batch in ['5workers_rand0','10workers_rand0','15workers_rand0','20workers_rand0','25workers_rand0','30workers_rand0']:
#     batch='30workers_rand0'
    Nworker = int(batch.split('workers')[0])
    batch_num = int(batch.split('rand')[-1])
    for objid in object_lst:
        print "batch: {}; obj{}".format(batch,objid)
        mega_mask = pkl.load(open('pixel_em/{}/obj{}/mega_mask.pkl'.format(batch,objid)))
        voted_workers_mask = pkl.load(open('pixel_em/{}/obj{}/voted_workers_mask.pkl'.format(batch,objid)))
        unique_tile_values = np.unique(mega_mask)
        # Tile loop 
        for tile_value in unique_tile_values[1:]: #exclude 0
            blobs = mega_mask==tile_value
            blobs_labels = measure.label(blobs,background=0)
            for tidx in np.unique(blobs_labels)[1:]:
                tile = blobs_labels==tidx
                worker_lst = list(set([item for sublist in voted_workers_mask[tile] if sublist!=0 for item in sublist]))
                for worker_id in worker_lst:
                    worker_tile_tbl.append([batch,objid,worker_id,tidx])

In [14]:
worker_tile_tbl = pd.DataFrame(worker_tile_tbl,columns=["batch","objid","worker id","tile id"])
dedup_worker_tile_tbl = worker_tile_tbl.drop_duplicates()
dedup_worker_tile_tbl.to_csv("all_pixel_worker_tile_info.csv")

``all_worker_response_info.csv`` and ``all_object_info.csv`` unchanged, ``all_object_batch_info.csv`` is only slightly affected